<a href="https://colab.research.google.com/github/jeawoo0594/programmers_project/blob/main/submit_pgrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

In [ ]:
train = pd.read_csv('/content/drive/My Drive/prgs/train.csv')
job_companies = pd.read_csv('/content/drive/My Drive/prgs/job_companies.csv')
job_tags = pd.read_csv('/content/drive/My Drive/prgs/job_tags.csv')
tags = pd.read_csv('/content/drive/My Drive/prgs/tags.csv')
test_job = pd.read_csv('/content/drive/My Drive/prgs/test_job.csv')
user_tags = pd.read_csv('/content/drive/My Drive/prgs/user_tags.csv')

In [ ]:
job_companies['companySize'] = job_companies['companySize'].fillna(0)
job_companies['companySize']

In [ ]:
job_com = job_companies[['jobID','companySize']]
companySize = []
# job_com
for tr in train['jobID'].values:
    
    for job,com in job_com.values:
        
        if tr == job:    
            
            companySize.append(com)
        else:
            pass
train['companySize'] = companySize
print(train)

In [ ]:
change_dic = {'1-10': 1, '11-50':2,'51-100':3,'101-200':4,'201-500':5,'501-1000':6,'1000 이상': 7}
train = train.replace({'companySize':change_dic })
train

In [ ]:
train = pd.get_dummies(data = train, columns = ['companySize'], prefix = 'companySize')

In [ ]:
train

In [ ]:
# jobtags = []
# # job_com
# for job,tag in job_tags.values:
    
#     for tai,key in tags.values:
        
#         if tag == tai:    
            
#             jobtags.append(key)
#         else:
#             pass
# job_tags['keyword'] = jobtags
# print(job_tags)

In [ ]:
# usertags = []
# # job_com
# for job,tag in user_tags.values:
    
#     for tai,key in tags.values:
        
#         if tag == tai:    
            
#             usertags.append(key)
#         else:
#             pass
# user_tags['keyword'] = usertags
# print(user_tags)

In [ ]:
# same_key = []
# for job,tag in zip(job_tags['jobID'],job_tags['keyword']):
    
#     for use,tad in zip(user_tags['userID'],user_tags['keyword']):
        
#         if tag == tad:
#             semi_same = []
#             semi_same.append(use)
#             semi_same.append(job)
#             semi_same.append(tad)
#             same_key.append(semi_same)
#         else:
#             pass
# same_key

In [ ]:
# new_key = []
# for v in same_key:
#     if v not in new_key:
#         new_key.append(v)
# print(new_key)

In [ ]:
# new_key

In [ ]:
# keyword_tr = []
# for user, job,k in new_key:
#     for use,jo in zip(train['userID'],train['jobID']):
#         if user == use and job == jo:
#             keyword_tr.append(k)
# train['keyword'] = keyword_tr
# train
# columns = ['userID','jobID','keyword']
# key_df = pd.DataFrame(new_key, columns=columns)
# key_df = pd.get_dummies(data = key_df, columns = ['keyword'], prefix = 'keyword')
# key_df

In [ ]:
target = train['applied']
train_data = train.drop('applied', axis=1)
train_data

In [ ]:
user_code = []
user_con =[]
count = 1
for cod in train_data['userID']:
    # print(cod)
    
    if cod not in user_code:

        user_code.append(cod)
        user_con.append(count)
        count += 1
    # else:
    #     user_code.append(count)
    #     user_con.append('overcount')

print(f'usercode: {user_code} \n usercount: {user_con}')

In [ ]:
user_dic = dict(zip(user_code,user_con))
user_dic


In [ ]:
# for data in train_data:
train_data['userID'] = train_data['userID'].map(user_dic)
train_data

In [ ]:
job_code = []
job_con =[]
count_j = 1
for job in train_data['jobID']:
    # print(cod)
    
    if job not in job_code:

        job_code.append(job)
        job_con.append(count_j)
        count_j += 1
    # else:
    #     user_code.append(count)
    #     user_con.append('overcount')

print(f'jobcode: {job_code} \n jobcount: {job_con}')

In [ ]:
job_dic = dict(zip(job_code,job_con))


In [ ]:

train_data['jobID'] = train_data['jobID'].map(job_dic)
train_data

In [ ]:
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits =5, shuffle=True, random_state = 12)
xgb = XGBClassifier()
xgb_param_grid = {'n_estimators':[100,200,300,400,500],
                 'learning_rate': [0.1, 0.01, 0.05, 0.2],
                 'max_depth': [4,8,12,16]}
xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid,cv=5,scoring='accuracy', n_jobs=-1,verbose=1)
xgb_grid.fit(train_data,target)
scores_df = pd.DataFrame(xgb_grid.cv_results_)
print(scores_df)

In [ ]:
print(f'best_param: {xgb_grid.best_params_}')
print(f'best_acc : {xgb_grid.best_score_}')

In [ ]:
from sklearn.svm import SVC
clf = SVC()
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
import numpy as np
print(np.mean(score))

In [ ]:
from lightgbm import LGBMClassifier,plot_importance
lgb = LGBMClassifier(feature_fraction = 0.8,num_leaves = 80)
lgb_param_grid = {'n_estimators':[100,300,500],
                 'learning_rate': [0.1, 0.01, 0.03],
                 'max_depth': [4,8,15,50],
                  'feature_fraction': [0.8,0.9]
                  }
lgb_grid = GridSearchCV(lgb, param_grid = lgb_param_grid,cv=5,scoring='accuracy', n_jobs=-1,verbose=1)
lgb_grid.fit(train_data,target)
scores_lgb = pd.DataFrame(lgb_grid.cv_results_)
print(scores_lgb)

In [ ]:
print(f'best_param: {lgb_grid.best_params_}')
print(f'best_acc : {lgb_grid.best_score_}')

In [ ]:
# test_job
companySize_test = []
# job_com
for tr in test_job['jobID'].values:
    
    for job,com in job_com.values:
        
        if tr == job:    
            
            companySize_test.append(com)
        else:
            pass
test_job['companySize'] = companySize_test
print(test_job)

In [ ]:
test_job = test_job.replace({'companySize':change_dic })
test_job

In [ ]:
test_job = pd.get_dummies(data = test_job, columns = ['companySize'], prefix = 'companySize')
test_job

In [ ]:
test_job['userID']

In [ ]:
user_code_test = []
user_con_test =[]
count_t = 1
for uct in test_job['userID']:
    # print(uct)
    
    if uct not in user_code_test:

        user_code_test.append(uct)
        user_con_test.append(count_t)
        count_t += 1
    # else:
    #     user_code.append(count)
    #     user_con.append('overcount')

print(f'usercode: {user_code_test} \n usercount: {user_con_test}')
# test_job['userID'] = test_job['userID'].map(user_dic)
# test_job['jobID'] = test_job['jobID'].map(job_dic)
# test_job.isnull().sum()

In [ ]:
usertest_dic = dict(zip(user_code_test,user_con_test))
# user_dic
test_job['userID'] = test_job['userID'].map(usertest_dic)
test_job

In [ ]:
job_code_t = []
job_con_t =[]
jobcount_t = 1
for jobs in test_job['jobID']:
    # print(cod)
    
    if jobs not in job_code_t:

        job_code_t.append(jobs)
        job_con_t.append(jobcount_t)
        jobcount_t += 1
    # else:
    #     user_code.append(count)
    #     user_con.append('overcount')

print(f'jobcode: {job_code_t} \n jobcount: {job_con_t}')

In [ ]:
jobtest_dic = dict(zip(job_code_t,job_con_t))
# user_dic
test_job['jobID'] = test_job['jobID'].map(jobtest_dic)
test_job

In [ ]:
test_job.isnull().sum()

In [ ]:
prediction = lgb_grid.predict(test_job)
prediction

In [ ]:
submission = pd.DataFrame({
    'applied': prediction
})
submission.to_csv('submission_prgs.csv',index=False)
!ls

In [ ]:
from google.colab import files
files.download('submission_prgs.csv')